In [ ]:
import tensorflow as tf 
from tensorflow import keras

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
class model(tf.keras.Model):
  def __init__(self):
    super().__init__()
    
    
    self.fc1 = keras.layers.Dense(784, activation = 'relu')
    
    self.fc2 = keras.layers.Dense(64, activation = 'relu')
    self.fc3 = keras.layers.Dense(64, activation = 'relu')
    self.fc4 = keras.layers.Dense(10, activation='softmax')
    self.optimizer = tf.keras.optimizers.Adam()
    self.loss = tf.keras.losses.SparseCategoricalCrossentropy()

  def __call__(self, input_tensor):
    x  = self.fc1(input_tensor)
    x = self.fc2(x)
    x = self.fc3(x)
    x = self.fc4(x)
    return x

  

In [ ]:
train_data, info = tfds.load("fashion_mnist", split = 'train', with_info = True)
test_data = tfds.load('fashion_mnist', split= 'test')


In [ ]:
class_names =   ['tshirt', 'pants', 'pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'AnkleBoots']

In [ ]:
def formate_img(data):
  img = data['image']
  img = tf.reshape(img, [-1])
  img = tf.cast(img, 'float32')
  img = img/255.0
  return img, data['label']

In [ ]:
train_data  = train_data.map(formate_img)
test_data = test_data.map(formate_img)

In [ ]:
batch_size = 64
train = train_data.shuffle(buffer_size = 1024).batch(batch_size)
test = test_data.batch(batch_size)

In [ ]:
model = model()
train_acc_metrics = tf.keras.metrics.SparseCategoricalAccuracy()
val_acc_metrics = tf.keras.metrics.SparseCategoricalAccuracy()

In [ ]:
def perform_validation():
  losses = []
  for x_val, y_val in test:
    val_preds = model(x_val)
    val_loss = model.loss(y_val, val_preds)
    losses.append(val_loss)
    val_acc_metrics(y_val, val_preds)
  return losses

In [ ]:
import numpy as np 

for epoch in range(2):
  train_loss = 0 
  train_n = 0

  epoch_losses_train = []
  epoch_losses_val = []
  for step, (x_batch_train, y_batch_train) in enumerate(train):

    with tf.GradientTape() as tape: 
      predictions  = model(x_batch_train)
      loss = model.loss(y_batch_train, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)
    model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))


    train_acc = train_acc_metrics.result()
    print(train_acc)
    losses_val = perform_validation()
    val_acc =   val_acc_metrics.result()

    losses_train_mean = np.mean(loss)
    losses_val_mean = np.mean(losses_val)
    epoch_losses_val.append(losses_val_mean)
    epoch_losses_train.append(losses_train_mean)
    print(epoch, train_acc)
    train_acc_metrics.reset_states()
    val_acc_metrics.reset_states()
  